In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re 
from tqdm import tqdm_notebook as tqdm

"""
集めるデータごとに関数を作成する。→ gather_information()で全部集める。
URLにアクセスする系は関数だとその度アクセスしてしまうので関数の中に組み込むのはよくない。
"""

In [2]:
#J1各チームの日程・結果にアクセスするためのurlリスト。
html_all = urlopen("https://soccer.yahoo.co.jp/jleague/teams/j1")
soup_all = BeautifulSoup(html_all, "html.parser")
team_url = soup_all.find_all(href = re.compile("^/jleague/teams/schedule"))
team_url_list= []
for a in team_url:
    team_url_list.append("https://soccer.yahoo.co.jp" + a.get("href"))

In [3]:
#J1チームリストの作成
team_name_list = []
for i in tqdm(team_url_list):
    html = urlopen (i)
    soup = BeautifulSoup(html, "html.parser")
    team_name = soup.find_all(class_ = "name")
    for i in team_name:
        team_name = i.text
        team_name_list.append(team_name)
team_name = ""

In [4]:
team_name_list

['北海道コンサドーレ札幌',
 'ベガルタ仙台',
 '鹿島アントラーズ',
 '浦和レッズ',
 '柏レイソル',
 'FC東京',
 '川崎フロンターレ',
 '横浜F・マリノス',
 '湘南ベルマーレ',
 '清水エスパルス',
 'ジュビロ磐田',
 '名古屋グランパス',
 'ガンバ大阪',
 'セレッソ大阪',
 'ヴィッセル神戸',
 'サンフレッチェ広島',
 'サガン鳥栖',
 'V・ファーレン長崎']

In [5]:
def score(soup):
    #各試合のスコアのTag
    score_tag = soup.find_all(href =  re.compile("^/jleague/game/score/"))
    #Tagからスコアをget
    score_list = []
    for v in score_tag:
        score_list.append(v.text)
    #文字化け？を修正するため一旦文字列にする。
    score_list_str = ",".join(score_list).replace("\xa0","")
    score_list = score_list_str.split(",")
    return(score_tag, score_list)

In [6]:
#結果を格納しているタグを発見してテキストにする。
def results(soup):
    results = soup.find_all(class_="textC")
    #勝敗だけ抽出。
    results_list = []
    for v in results:
        if (v.text == "△" or v.text == "●" or  v.text == "○" ):
            results_list.append(v.text)
    #リストの勝敗を数値に変換。
    results_list = ["2" if "△" in s  else s for s in results_list]
    results_list = ["1" if "○" in s  else s for s in results_list]
    results_list = ["0" if "●" in s  else s for s in results_list]
    return(results_list)

In [7]:
def home_review(score_tag, teamname):
    #Home/Away
    home_away_url = []
    home_away_list = []
    #review
    match_review = []
    #試合詳細のURLをリスト化
    for a in score_tag:
        home_away_url.append("https://soccer.yahoo.co.jp"  + a.get("href"))
    #リスト化されたURLからHOME/AWAY情報を抽出
    for i in tqdm(home_away_url, teamname):
        home_away_html = urlopen(i)
        home_away_soup = BeautifulSoup(home_away_html, "html.parser")
        if str(home_away_soup.title.contents[0]).split()[-7] == teamname :
            home_away_list.append(1)
        else:
            home_away_list.append(0)
        review_test = home_away_soup.find("p",class_="", text=re.compile("。")).text
        pattern = re.compile("掲載")
        matchObj = pattern.search(review_test)
    #掲載　という文字が入っていたらレビューではないので空にする。
        if matchObj:
            match_review.append("Nah")
        else:
            match_review.append(review_test)
    return(home_away_url, home_away_list, match_review)

In [8]:
###xに入るのは数字
###team x の情報を集める関数。
def gather_information(x):
    #各チームにアクセス。
    html = urlopen(team_url_list[x])
    soup = BeautifulSoup(html, "html.parser")
    #チーム名
    teamname = team_name_list[x]   
    df_score = score(soup)
    score_tag = df_score[0]
    df_results = results(soup)
    df_home_review = home_review(score_tag, teamname)
    #DataFrame→CSV にして出力
    df = pd.DataFrame({"results" : df_results,
                                     "home" : df_home_review[1],
                                     "score" : df_score[1],
                                       "match review" : df_home_review[2]
                                  })
    df.to_csv("/Users/yamadaikuya/Desktop/pythonista/toto/results/"+teamname+".csv",sep=",")

In [10]:
gather_information(0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yamadaikuya/pythonista/toto/results/北海道コンサドーレ札幌.csv'

In [65]:
#全てのチームに関して情報を集める。
def gather_all_information():
    for i in tqdm(range(len(team_url_list)),"Overall Progress"):
        gather_information(i)

In [66]:
gather_all_information()

In [15]:
#csvからdfに戻す。
def show_dataframe(i):
    df_return = pd.read_csv("/Users/yamadaikuya/pythonista/toto/results/"+team_name_list[i]+".csv", 
                      index_col=[0])
    df_return_latest = df_return[-1:] #最新のが見たいときはこっち
    pd.options.display.max_colwidth = 1000 #最大文字数の変更
    pd.options.display.max_rows = 1000 #最大行数の変更
    return(display(df_return))

In [77]:
#データフレームの扱い。カラムを増やす。
pd.options.display.notebook_repr_html = True
pd.options.display.max_colwidth = 1000 #最大文字数の変更
pd.options.display.max_rows = 1000 #最大行数の変更
df_return = pd.read_csv("/Users/yamadaikuya/pythonista/toto/results/"+team_name_list[i]+".csv", 
                  index_col=[0])
df_return["A"]=df_return["score"]
df_return

,results,home,score,match review,A
0,0,0,2-1,昨季圧倒的なチーム力でJ2を制し、2年ぶりにJ1に帰還した湘南。開始早々に松田の力強い突破から新戦力のイジョンヒョプが先制点を奪い、そのままゲームを優位に進めるかと思われた。ところが、前半16分に追い付かれて以降、相手のコンパクトな守備を前に攻撃陣が沈黙。重苦しい空気が漂ったまま時間が経過してしまう。だが、そんな状況を打破したのは得意とするセットプレーだった。絶好の位置でFKを得ると、秋野のキックから最後は石川が押し込んで後半35分に勝ち越しに成功。少ないチャンスを確実に生かし、勝利を収めた。,2-1
1,2,1,2-2,J1での記念すべき初のホーム開幕戦を迎えた長崎は、序盤から積極的な姿勢を見せる。前半2分に澤田が豪快にネットを揺らし、さい先よく先制に成功。同35分には加入後初先発を果たした鈴木にもゴールが生まれ、最高の形で試合を折り返す。しかし、後半15分に失点を喫してしまい、流れは完全に鳥栖へ傾く。投入したチェキュベックを最終ラインに配置し、逃げ切りを図るものの、同39分にセットプレーから同点に追い付かれてしまう。雨が降る中でもトラスタに駆け付けた大勢のサポーターに、勝利をプレゼントすることはできなかった。,2-2
2,2,0,2-2,Nah,2-2
3,2,1,1-1,J1初勝利を挙げたい長崎。浦和にボールを支配される時間帯が続くが、ファンマのキープ力を起点にして攻撃を仕掛ける。前半44分には、翁長が遠藤に倒されて得たPKをこのゲームでJ1通算100試合出場を達成した鈴木が確実に決め、リードを奪って前半を終了。後半に入っても守ってカウンター攻撃を仕掛けるスタイルは変えずに粘り強く戦ったものの、後半31分に左CKからズラタンに同点弾を決められてしまう。その後も勝利を目指してチャンスは創出したが、最後まで勝ち越し点を挙げられず。前節に続いて勝点3を逃してしまった。,1-1
4,1,1,2-1,Nah,2-1
5,0,0,2-1,今季初勝利を狙う札幌。パスをつなぎながら両サイドからのクロスとジェイの高さを起点にして攻撃を仕掛けるが、得点を奪えぬまま試合を折り返す。後半に入ると、チャナティップが躍動。後半8分に体の強さを生かしてDFをかわし、ジェイに縦パスを送ると、ジェイがこれを決めて先制する。同点で迎えた同49分には、ジェイが頭で折り返したボールに反応して劇的なヘディングシュートを決め、今季初の勝点3に大きく貢献した。対する長崎は、一時は同点に追い付いたが、逆転ゴールを奪えず。次節でJ1初勝利を挙げたい。,2-1
6,0,0,1-0,今季いまだ無敗と好調を維持する仙台がまたしても「ウノゼロ」の勝利を挙げた。立ち上がりこそ相手の素早いプレスに加えて鈴木、ファンマの裏への抜け出しに手を焼き、押し込まれる時間帯が続いたが、それでも耐えられるのが今季のホームチーム。しっかり相手の攻撃をシャットアウトすると、今季初先発の西村が相手のクリアミスを見逃さず、ユアスタを沸かす。その後もバランスよくゲームをコントロールし、相手の攻撃を抑え込んだ。一方の長崎はチャンスをモノにできず。またしてもJ1初勝利はお預けとなった。,1-0
7,1,1,3-2,Nah,3-2
8,0,1,2-5,長崎は開始直後の失点で出はなをくじかれると、その後も自陣の左サイドのスペースを活用されてピンチを招くなど、ペースをつかめない。ボールを保持した状態であっても、味方のパスを引き出す動きが乏しく、ロングボールに頼ることが多くなり、個々の奮闘に頼るシーンが頻出。セットプレーとショートカウンターで2点を奪うものの、リーグ戦初勝利には届かなかった。対するFC東京は、個人技や連係で一枚上手であることを見せつける。ディエゴオリヴェイラがハットトリックを達成し、3連勝を飾った。,2-5
9,1,0,0-1,長崎が歴史的勝利を手にした。序盤からお互いに攻めあぐねる状況だったが、鈴木が均衡を破る先制弾を奪取。1点のリードを保って前半を折り返すと、後半はホームチームが前線の選手を3枚投入して攻勢を強める。クリスラン、ミッチェルデューク、北川に度々攻め込まれるが、運動量で引けを取らないアウェイチームは何度もその攻撃をはね返した。終盤のパワープレーを全員守備でしのぐと、ここで試合終了のホイッスル。アイスタでJ1史上初の勝点3を手にし、選手、スタッフ、サポーターが一体となって歓喜に沸いた。,0-1


In [64]:
for i in range(len(team_name_list)):
    show_dataframe(i)

,results,home,score,match review
0,0,0,1-0,復活を目指す広島と、さらなるステップアップを見据える札幌。ともに新しい指揮官を迎えて臨んだ一戦は、充実の内容を見せた。広島は柏と川辺の両サイドを中心に主導権を握ると、前半28分にJリーグデビュー戦のティーラシンが先制点を奪う。その後は守勢に回る時間が続いたが、ボールホルダーへのプレスを徹底。90分間走り抜いて勝点3を手に入れた。一方、良い攻めを見せたものの最後の精度が足りず1点に泣いた札幌。だが、ピッチを広く使って攻め立てる「ミシャサッカー」の浸透ぶりを随所に見せ、今後への期待を持たせた。
1,2,0,3-3,ホームのC大阪は前半からプレスの強度を高く保ち、奪えばすぐにカウンターへ持ち込む。パス主体の攻撃でチャンスをうかがう札幌のミスを誘ってゲームの主導権を握ると、今季から加入した高木の得点を含む2点を奪い、試合を折り返す。だが、リードを得たことでディフェンスに対する意識が強くなり、最終ラインをずるずると下げてしまう。時間を追うごとにゴール前のマークが甘くなっていくと、後半だけでアウェイチームに3ゴールを献上。勝点1を分け合ったものの、守備面のもろさを露呈して今後の連戦に不安が残る結果となった。
2,0,0,3-0,Nah
3,0,1,1-3,ポゼッション対ショートカウンターと、互いのスタイルが明確に出た一戦となった。今季初のホームゲームに臨んだ札幌は、清水のコンパクトなブロックを逆手に取るように、ピッチを広く使った攻撃を見せる。フリーで受けて正確なクロスを供給する三好、相手のギャップに顔を出し続けるチャナティップの2人が、両ウイングバックと連動して守備網を切り裂いた。しかし、清水の素早い切り替えに守備陣が対応し切れず、時間とともに失点を重ねることに。チームの形はともに表現できただけに、その完成度の差が結果として如実に表れてしまった。
4,0,1,0-1,Nah
5,1,1,2-1,今季初勝利を狙う札幌。パスをつなぎながら両サイドからのクロスとジェイの高さを起点にして攻撃を仕掛けるが、得点を奪えぬまま試合を折り返す。後半に入ると、チャナティップが躍動。後半8分に体の強さを生かしてDFをかわし、ジェイに縦パスを送ると、ジェイがこれを決めて先制する。同点で迎えた同49分には、ジェイが頭で折り返したボールに反応して劇的なヘディングシュートを決め、今季初の勝点3に大きく貢献した。対する長崎は、一時は同点に追い付いたが、逆転ゴールを奪えず。次節でJ1初勝利を挙げたい。
6,2,0,0-0,前半の鹿島は辛抱の時間が続く。開始直後の都倉のシュートを皮切りに、札幌の連動したスプリントで自陣深くにボールを運ばれる。特に駒井にサイドを破られて肝を冷やす場面が続出し、ディフェンスラインの集中力が問われる展開となる。見せ場は作るが、それ以上のピンチを迎え、後半の選手交代でも流れは変わらず。スコアレスドローの決着となった。対して札幌は、リスクを背負った攻撃で優位に進め、カシマでのリーグ戦初勝利は目前に迫ったが、相手の守備陣の粘りに遭って得点には至らず。鬼門での初の勝点は1となった。
7,0,0,2-1,Nah
8,1,1,3-0,カウンターを狙う札幌とボールを回す名古屋という互いの狙いが前半から交錯する試合展開。ここで持ち味を発揮したのはホームチームだった。右サイドで駒井と三好を中心にカウンターからチャンスを作ると、相手はCKに逃れることに精いっぱい。前半26分にはそのCKから先制点を奪うと、ここから一気に主導権は札幌へ。後半に入っても勢いはとどまらず、エース都倉がバイシクルで豪快に追加点をたたき込んだ。最後は古巣対決となった櫛引のオウンゴールもあり、3点を奪っての完勝。リーグ戦2試合ぶりの勝点3を手にした。
9,1,1,1-0,平日にも関わらず、10000人を超える観衆が駆け付けた一戦。札幌は、前半から都倉や三好を中心に相手ゴールを脅かした。だが、決定的なチャンスを生かせず、0－0のまま試合を折り返す。後半に入っても、勢いそのままに攻め続けるが、ネットを揺らせず、アディショナルタイムに突入する。このまま終了かと思われたが、後半46分に都倉が均衡を打破する貴重な1点を奪取。札幌ドには多くのサポーターの歓声が響き渡り、選手にとっても観客にとってもうれしい勝点3となった。


,results,home,score,match review
0,1,1,1-0,前半のホームチームはつなぎのミスの連発でチャンスをつぶしては、ピンチを迎え続けた。それでも、じれることなく相手の攻撃をことごとくシャットアウトすると、後半8分に古林からの鋭いクロスボールをDFの板倉が高い打点から頭でたたき込み、先制に成功。リードという「盾」を得てからは、ため息すら出てしまうような連動性を披露してフィールドの随所で数的優位を作り、分厚い攻撃から何度もゴール前へ迫った。終盤には相手の波状攻撃に苦しんだものの、耐えしのいでタイムアップ。仙台は4季連続で開幕戦の完封勝利を達成した。
1,1,0,0-1,FC東京は序盤から鋭いプレスを掛けて何度もチャンスを迎えるも、仙台の強固な3バックを崩し切れない。逆に、疲れからか動きが徐々に鈍くなり、相手に自由にボールを動かされて先制を許してしまう。ビハインドを負ったホームチームは久保を投入すると、16歳の俊英は卓越したサッカーセンスを存分に発揮し、リズムをもたらす。しかし、久保が徹底的にマークされると、攻撃はまたしても停滞し、なすすべもなく0－1で試合終了。抜群の相性を誇っていた仙台を相手に、味スタでの初勝利をやすやすとプレゼントしてしまった。
2,2,1,1-1,Nah
3,2,1,1-1,前半は互いに思うような攻撃の形を作れない展開。ホームチームがボールを保持する戦況となるが、ワンチャンスを神戸に決められて今季の初失点を喫する。追う展開で決定機を作れないまま前半を折り返す。後半に入って西村を投入すると、システムを変更。前半とは打って変わってシュートで終わる場面が増えていく。波状攻撃を見せていく中で、後半41分にシュートのこぼれ球に石原が詰め、待望の同点弾をゲット。連勝こそストップしたが、ホームで貴重な勝点1を手にした。
4,2,0,0-0,Nah
5,2,0,1-1,清水は開始早々に金子のスーパーゴールで先制し、さい先の良いスタートを切る。対する仙台はボールを左右に揺さぶりながらチャンスをうかがうも、清水の強固な守備ブロックを前に苦戦。逆に、清水はボールを奪うと、すかさず前線のクリスランへと預けるが、このクリスランには昨季の「盟友」たちが厳しくマークに付き、自由を与えない。まさに攻守が引き締まった一進一退の展開が続く中、徐々に仙台ペースになった後半25分に同点弾を奪われる。それでも、試合の大勢は以後も変わらず、「無敗対決」はドローでの決着となった。
6,1,1,1-0,今季いまだ無敗と好調を維持する仙台がまたしても「ウノゼロ」の勝利を挙げた。立ち上がりこそ相手の素早いプレスに加えて鈴木、ファンマの裏への抜け出しに手を焼き、押し込まれる時間帯が続いたが、それでも耐えられるのが今季のホームチーム。しっかり相手の攻撃をシャットアウトすると、今季初先発の西村が相手のクリアミスを見逃さず、ユアスタを沸かす。その後もバランスよくゲームをコントロールし、相手の攻撃を抑え込んだ。一方の長崎はチャンスをモノにできず。またしてもJ1初勝利はお預けとなった。
7,1,1,3-0,Nah
8,0,0,1-0,開始早々にDFのわずかな隙を突いた「仙台キラー」の興梠が落ち着いてシュートを流し込み、浦和は先制に成功。だが、前半の半ばあたりから仙台に中盤を掌握され始め、徐々に劣勢に。ピッチの随所を動き回るボールと相手を捕まえ切れず、左右に揺さぶられ続けた。それでも、この日もフィニッシュの迫力を欠いたアウェイチームを相手に、最後の局面ではシンプルな守備を徹底。この割り切った策が功を奏し、虎の子の1点を守り抜いた浦和がホームでは昨夏以来の白星を手にした。
9,1,0,2-3,ガブリエルシャビエルの欠場、ジョーのベンチスタートと、個の力が抜けたことで攻撃の迫力を欠いた名古屋は、前半のうちにサイド攻撃から2失点。たまらずジョーを投入するも、苦しい展開のまま試合を折り返す。だが、後半に入ると、ジョーがボールを多く触ってリズムを生み出し、自らも開幕戦以来のゴールを奪う。しかし、一瞬の隙を突かれて再び失点。終盤にPKで2点目を奪って追いすがるも、反撃はここまで。これで4連敗となった。一方の仙台は、今季初の複数得点で勝利。前節の初黒星からしっかり立て直し、勝点3を積み上げた。


,results,home,score,match review
0,2,1,1-1,Nah
1,1,0,1-2,Nah
2,2,0,0-0,ドイツから内田が8季ぶりに復帰するなど、連覇を逃した昨季より一段と選手層が厚くなった鹿島に対し、清水は勇猛果敢に立ち向かった。序盤から北川、金子、石毛らの若手選手たちが、ボール保持力の高いアウェイチームの中盤や最終ラインにプレスを掛け、奪った後は素早く攻撃に切り替える。徐々に相手を押し込むと、前半40分にPKを獲得。しかし、キッカーを務めた新加入のクリスランのシュートはGKに読まれてしまう。それでも、チャンスを逸した悪いムードを後半へ引きずらず、守備では見事な統率を見せて引き分けに持ち込んだ。
3,1,1,1-0,鹿島は序盤からサイド主体の攻撃でG大阪を手玉に取る。金崎を中心としたリズムの良いパス回しが敵陣を切り裂くも、肝心のフィニッシュが雑になり、ゴールが遠い。後半も押し込む時間帯が続くが、東口の分厚い壁を破れずに時計の針が進んでいく。しかし、途中出場の鈴木がチームを救う。個人技でチャンスを作り、金崎の先制点をお膳立て。ホーム初戦を白星で飾り、連戦へ向けてさい先の良いスタートを切った。一方のG大阪は後半から入った中村が流れを変えるプレーを見せたものの、中盤のミスが多く終始劣勢に。点差以上の完敗を喫した。
4,1,0,0-2,Nah
5,0,1,0-1,互いにインテンシティーが高く、攻守が目まぐるしく入れ替わる展開が繰り広げられる。その中で、鹿島は安西、広島は川辺がとりわけ目立った働きを披露。両チームともにチャンスを作りつつも、スコアレスで前半を終えた。後半に入り、ミスを突かれて失点を喫したホームチームは、徐々に攻撃の圧力を強めていく。だが、ゴール前を固く閉ざす相手を崩せない。結局「常勝軍団」のお株を奪われるような戦い方を最後まで遂行され、近年得意としてきた広島戦で勝点を落としてしまった。
6,2,1,1-1,Nah
7,1,0,0-1,互角の戦いを見せた鳥栖だったが、鹿島のエース金崎のゴールに沈んだ。前半から両チームともに球際が激しく、チャンスの応酬となる。鳥栖はショートカウンターの形から失点を喫したが、後半はビクトルイバルボにロングボールを集めることで多くの決定機を創出。何度もゴールに迫るが、相手守備陣の好守に阻まれ、あと一歩届かなかった。一方の鹿島は相手にペースを握られる時間帯が続くも、チーム全体で耐えて勝点3を獲得。特に、ビクトルイバルボを抑え込んだ昌子と植田のパフォーマンスは素晴らしく、代表戦に弾みをつけた。
8,2,1,0-0,前半の鹿島は辛抱の時間が続く。開始直後の都倉のシュートを皮切りに、札幌の連動したスプリントで自陣深くにボールを運ばれる。特に駒井にサイドを破られて肝を冷やす場面が続出し、ディフェンスラインの集中力が問われる展開となる。見せ場は作るが、それ以上のピンチを迎え、後半の選手交代でも流れは変わらず。スコアレスドローの決着となった。対して札幌は、リスクを背負った攻撃で優位に進め、カシマでのリーグ戦初勝利は目前に迫ったが、相手の守備陣の粘りに遭って得点には至らず。鬼門での初の勝点は1となった。
9,2,0,2-2,Nah


,results,home,score,match review
0,2,0,1-1,アタッキングサードでミスを連発したホームチームと、アイデアの乏しい単調な攻撃に終始したアウェイチームの対戦はロースコアでのドローとなった。FC東京は新加入のディエゴオリヴェイラが屈強な肉体を生かしたポストプレーで起点となるも、味方がボールを受けられずにかみ合わない場面が続く。一方の浦和も、両サイドからの仕掛けに頼るばかりで「らしい」崩しを見せるシーンはほとんどなかった。FC東京は途中出場の久保が攻撃を一時的に活性化させるも、チームの停滞したムードは覆せず。1－1での決着は妥当な結果と言えよう。
1,0,1,1-2,浦和は立ち上がりからしっかりとパスを回してボールを握ると、右サイドからはマルティノスが、左サイドからは宇賀神が好機を演出する。前半終了間際には人数を掛けた攻撃を見せ、最後は青木がヘディングシュートを決めて先制。後半に入ってもアタッキングサードに持ち込む場面を多く作り、今季初勝利へサポーターの期待は高まった。しかし、追加点を奪えないことが明暗を分ける。少ない決定機を得点に結び付けた相手に逆転負けを喫した。対して耐える時間が長くなった広島は粘り強い守備を最後まで貫き、開幕2連勝を飾った。
2,1,0,1-4,Nah
3,2,0,1-1,J1初勝利を挙げたい長崎。浦和にボールを支配される時間帯が続くが、ファンマのキープ力を起点にして攻撃を仕掛ける。前半44分には、翁長が遠藤に倒されて得たPKをこのゲームでJ1通算100試合出場を達成した鈴木が確実に決め、リードを奪って前半を終了。後半に入っても守ってカウンター攻撃を仕掛けるスタイルは変えずに粘り強く戦ったものの、後半31分に左CKからズラタンに同点弾を決められてしまう。その後も勝利を目指してチャンスは創出したが、最後まで勝ち越し点を挙げられず。前節に続いて勝点3を逃してしまった。
4,0,1,1-4,Nah
5,0,1,0-1,浦和は両サイドバックが積極的に攻撃参加をし、早めにクロスを入れて相手の高いディフェンスラインの裏を突く。対して横浜FMは飯倉が時にはペナルティエリアを飛び出して守備に貢献し、相手の攻撃をうまく網にかける。互いの良さが前面に出た展開だっただけに、明暗を分けたのは決定力の高さだった。チャンスをモノにしたストライカーはアウェイチームのウーゴヴィエイラ。ペナルティエリア内でしっかり仕事をこなし、チームを今季初勝利に導いた。一方、浦和はホームというアドバンテージを生かせず、勝点3を献上してしまった。
6,0,0,2-1,エースがようやく覚醒の時を迎え、磐田が逆転勝利を収めた。序盤にPKで失点を喫した「サックスブルー」。だが、徐々にダブルボランチが幅広く攻撃に絡んで好機を作る。西川の好守に苦しめられるも、前半の終了間際に川又の巧みなシュートで同点に。ここまで無得点の川又はこれで上昇気流に乗り、後半は力強いポストプレーで前線に君臨。そして、逆転弾を決めたのも背番号20だった。積極的に打ったシュートが相手に当たって決まり、2－1で勝利。15連戦の初戦を制する立役者となったストライカーに、サポーターの歓声が降り注いだ。
7,2,0,0-0,Nah
8,1,1,1-0,開始早々にDFのわずかな隙を突いた「仙台キラー」の興梠が落ち着いてシュートを流し込み、浦和は先制に成功。だが、前半の半ばあたりから仙台に中盤を掌握され始め、徐々に劣勢に。ピッチの随所を動き回るボールと相手を捕まえ切れず、左右に揺さぶられ続けた。それでも、この日もフィニッシュの迫力を欠いたアウェイチームを相手に、最後の局面ではシンプルな守備を徹底。この割り切った策が功を奏し、虎の子の1点を守り抜いた浦和がホームでは昨夏以来の白星を手にした。
9,1,0,2-3,神戸はボールを保持し、パスを回して左サイドへ集める。橋本の果敢なオーバーラップから好機に結び付けるが、逆に空いた自陣のスペースを活用されてしまう。シンプルに速い攻撃を仕掛ける浦和に先制を許して折り返すと、後半はルーキーの佐々木が逆転ゴールを奪い、岩波の恩返し弾が飛び出すなど打ち合いの展開に。このまま引き分けかと思われたが、土壇場でマウリシオに決められて敗戦を喫した。対して連勝を挙げた浦和も、若い力が躍動。右サイドでスプリントを繰り返した橋岡のプレーは、生まれ変わったチームの象徴となった。


,results,home,score,match review
0,1,0,3-0,Nah
1,0,0,3-2,Nah
2,2,1,1-1,Nah
3,0,0,1-0,前半のホームチームはつなぎのミスの連発でチャンスをつぶしては、ピンチを迎え続けた。それでも、じれることなく相手の攻撃をことごとくシャットアウトすると、後半8分に古林からの鋭いクロスボールをDFの板倉が高い打点から頭でたたき込み、先制に成功。リードという「盾」を得てからは、ため息すら出てしまうような連動性を披露してフィールドの随所で数的優位を作り、分厚い攻撃から何度もゴール前へ迫った。終盤には相手の波状攻撃に苦しんだものの、耐えしのいでタイムアップ。仙台は4季連続で開幕戦の完封勝利を達成した。
4,1,1,2-0,柏は横浜FMの流動的な攻撃サッカーに苦しみ、選手をつかまえ切れずにボールを支配され続けてしまう。それでも、最後の局面では相手に自由を与えずにしのぐと、後半開始早々だった。DFの一瞬のズレを突いた小泉が巧みなワンツーから守備網をかいくぐり、先制点を奪取。ここからホームチームは息を吹き返したように攻勢に転じ、伊東やクリスティアーノらの快足FWが何度も相手の裏を突いてゴールへ迫った。終盤の波状攻撃にも耐え、手堅く白星をゲット。ACLでも苦戦の続いた「太陽王」は、ようやく戦績表に「快晴」の印を刻んだ。
5,1,1,1-0,Nah
6,2,1,1-1,公式戦2試合連続で完封勝利と上り調子の柏。開始早々に予想外の形でスコアが動く。高い位置でパスミスを拾った江坂から、最後はクリスティアーノが押し込んで先制点を獲得。さい先の良いスタートを切り、ゲームを優位に進めるかと思われた。しかし、以降はC大阪の猛攻に遭い、大半の時間を自陣で過ごす展開に。中村を中心とする懸命な守備で耐えしのぐが、後半15分にソウザの豪快な一撃で追い付かれてしまう。それでも、逆転弾は許さず。苦しみながらも粘り強く戦い抜き、勝点1を獲得した。
7,0,0,1-0,Nah
8,2,0,2-2,開幕3連敗中の悪い流れを断ち切りたかったG大阪だったが、前半8分、同18分と立て続けに江坂にネットを揺らされてしまう。それでも、失点直後の同21分にファンウィジョが1点を返すと、チームは息を吹き返す。市丸の負傷でボランチにポジションを移した遠藤がパスで攻撃のリズムを刻めば、途中出場の中村はドリブルでアクセントを付けて柏の守備陣を翻ろう。不安定な守備から何度かピンチを迎えたが、東口を中心に耐えしのぎ、ファンウィジョの得点で後半41分に同点に追い付く。課題と同時に収穫も多い引き分けとなった。
9,1,1,2-1,同じファーストネームを持つ「ジュンヤ」対決は、柏の伊東に軍配が上がった。スコアレスで迎えた後半に「IJ」こと伊東が先に輝きを放つ。後半23分に低い位置から得意のドリブルで一気に前線へと迫り、最後は左足を華麗に振り抜いて先制点をゲットした。だが、わずか6分後に、「TJ」こと田中も古巣相手に恩返し弾を見舞い、意地を見せる。それでも、この日の主役はホームのサポーターの大声援を受け、躍動したスピードスターだった。同41分に伊東がこぼれ球を押し込んで勝負あり。柏が接戦をものにし、勝点3をもぎ取った。


...

,results,home,score,match review
0,1,0,2-3,C大阪が富士ゼロックススーパーカップ初優勝を果たした。序盤からマークの激しさで相手を上回って主導権を握り、前半26分には先制に成功する。後半の頭にはGKのロングボールから素早く攻め込み、清武が追加点を奪取。流れを確かなものにすると、ヤンドンヒョンから高木という新加入の2人で3点目を挙げる。終了間際に軽率な対応から失点を喫したのは反省点だが、仕上がりの良さを見せて今季の初戦で戴冠に成功した。対する川崎Fは1点及ばず敗戦。昨季のリーグ王者は、またしても一発勝負の一戦での弱さを露呈する結果となった。
1,1,0,0-1,Nah
2,2,1,0-0,Nah
3,2,1,1-1,C大阪は序盤から横浜FMの強烈なプレッシングに対して受け身になり、ミドルサードで不用意なミスを連発。自らリズムを崩していくと、前半17分に下がり過ぎた最終ラインの隙を突かれて先制点を献上する。相手のスタイルに順応した後半は怒とうの攻撃を展開するも、クロス、シュートの精度を欠く場面が目立ち、同点に追い付くまでが精いっぱいだった。一方の横浜FMも新指揮官の下で魅力あるポゼッションサッカーを披露したが、C大阪の猛攻に耐え切れず、終盤に失点。元日決戦の再来となった一戦は悔しいドロー決着となった。
4,2,1,3-3,ホームのC大阪は前半からプレスの強度を高く保ち、奪えばすぐにカウンターへ持ち込む。パス主体の攻撃でチャンスをうかがう札幌のミスを誘ってゲームの主導権を握ると、今季から加入した高木の得点を含む2点を奪い、試合を折り返す。だが、リードを得たことでディフェンスに対する意識が強くなり、最終ラインをずるずると下げてしまう。時間を追うごとにゴール前のマークが甘くなっていくと、後半だけでアウェイチームに3ゴールを献上。勝点1を分け合ったものの、守備面のもろさを露呈して今後の連戦に不安が残る結果となった。
5,0,0,2-0,Nah
6,2,0,1-1,公式戦2試合連続で完封勝利と上り調子の柏。開始早々に予想外の形でスコアが動く。高い位置でパスミスを拾った江坂から、最後はクリスティアーノが押し込んで先制点を獲得。さい先の良いスタートを切り、ゲームを優位に進めるかと思われた。しかし、以降はC大阪の猛攻に遭い、大半の時間を自陣で過ごす展開に。中村を中心とする懸命な守備で耐えしのぐが、後半15分にソウザの豪快な一撃で追い付かれてしまう。それでも、逆転弾は許さず。苦しみながらも粘り強く戦い抜き、勝点1を獲得した。
7,2,0,2-2,Nah
8,0,0,2-0,神戸は序盤から主導権を握る。両サイドにボールを振り分けつつ、時には中央からミドルシュートを積極的に放ち、C大阪ゴールを脅かす。藤田の技ありゴールで先制に成功して試合を折り返すと、後半はまず守備に重点を置き、相手に決定機を作らせない。自陣に押し込まれる時間帯をしのぎ切り、最後はルーカスポドルスキが役者の違いを見せつけて勝負あり。リーグ戦今季初白星とJ1通算200勝を「神阪ダービー」で収めた。敗れたC大阪はこれで公式戦7戦未勝利。攻守ともに内容が良化する兆しはなく、暗雲は立ち込める一方だ。
9,1,1,2-1,桜前線の上昇から一歩遅れ、金鳥スタでもようやく「桜」が花開いた。今季初勝利をつかむべく、ヤンドンヒョンを前線の基点に据えたC大阪。序盤こそ主導権を握るも、時間が進むにつれ、湘南の運動量に押し込まれ始める。松田らのプレスに手を焼くだけでなく、守備陣も次々に走り込む相手を捕まえ切れず、苦しい展開に。しかし、ピッチに立ち込めた暗雲を振り払ったのは、交代で入った杉本の鮮やかな一撃だった。最後まで運動量を落とさなかった敵との我慢比べを制し、つかみ取った大きな白星を、今後のさらなる浮上の契機にしたい。


,results,home,score,match review
0,2,0,1-1,鳥栖は開始早々にPKを獲得すると、田川が冷静に沈めてさい先よいスタートを切る。リードを得て戦いやすくなったホームチームは堅いブロックを自陣に敷き、ボールを奪えば素早くカウンターを仕掛ける盤石のゲーム展開を披露する。だが、決定機を逃し続けていると、終盤は攻撃的な選手を次々と送り込んできたアウェイチームに対してディフェンスラインをずるずると下げてしまい、後半42分には痛恨の同点弾を献上。90分を通して優位に立つ時間が長かっただけに、ベアスタで勝点2を落とすもったいない開幕戦となってしまった。
1,0,1,2-4,今季ホーム初勝利を目指す神戸。自陣からパスをつないでチャンスをうかがうも、清水の強度の高いプレッシャーを前にして徐々にミスが多くなる。クリスランと北川にゴールを奪われて2点のリードを許すが、前半32分に田中が1点を返し、流れを得たまま試合を折り返す。後半開始直後からも主導権をつかむと、後半24分にチョンウヨンのFKがゴール左隅に決まり、同点に追い付く。この勢いで逆転ゴールを獲得したかったが、相手のカウンターを中心とした攻撃を止められずに失点を重ねて勝利を逃した。
2,2,1,2-2,Nah
3,2,0,1-1,前半は互いに思うような攻撃の形を作れない展開。ホームチームがボールを保持する戦況となるが、ワンチャンスを神戸に決められて今季の初失点を喫する。追う展開で決定機を作れないまま前半を折り返す。後半に入って西村を投入すると、システムを変更。前半とは打って変わってシュートで終わる場面が増えていく。波状攻撃を見せていく中で、後半41分にシュートのこぼれ球に石原が詰め、待望の同点弾をゲット。連勝こそストップしたが、ホームで貴重な勝点1を手にした。
4,1,0,0-2,Nah
5,1,1,2-0,神戸は序盤から主導権を握る。両サイドにボールを振り分けつつ、時には中央からミドルシュートを積極的に放ち、C大阪ゴールを脅かす。藤田の技ありゴールで先制に成功して試合を折り返すと、後半はまず守備に重点を置き、相手に決定機を作らせない。自陣に押し込まれる時間帯をしのぎ切り、最後はルーカスポドルスキが役者の違いを見せつけて勝負あり。リーグ戦今季初白星とJ1通算200勝を「神阪ダービー」で収めた。敗れたC大阪はこれで公式戦7戦未勝利。攻守ともに内容が良化する兆しはなく、暗雲は立ち込める一方だ。
6,0,0,2-1,同じファーストネームを持つ「ジュンヤ」対決は、柏の伊東に軍配が上がった。スコアレスで迎えた後半に「IJ」こと伊東が先に輝きを放つ。後半23分に低い位置から得意のドリブルで一気に前線へと迫り、最後は左足を華麗に振り抜いて先制点をゲットした。だが、わずか6分後に、「TJ」こと田中も古巣相手に恩返し弾を見舞い、意地を見せる。それでも、この日の主役はホームのサポーターの大声援を受け、躍動したスピードスターだった。同41分に伊東がこぼれ球を押し込んで勝負あり。柏が接戦をものにし、勝点3をもぎ取った。
7,1,0,0-3,Nah
8,1,0,0-1,「阪神ダービー」にふさわしい劇的な幕切れとなった。G大阪はルヴァンカップで結果を残した長沢を1トップに据えて臨むが、高い位置からのプレスに押され気味となってしまう。そんな中で、後半10分に迎えた決定機から状況が一転する。マテウスが本調子を発揮し、幾度も中央突破でチャンスを創出。徐々に前での迫力が増し、得点のにおいを感じさせた。だが、終了間際に勝負強さを見せられ、痛い敗戦を喫した。対する神戸は変化を付けられず、簡単なクロス一辺倒の後半だったが、ラストで連動した攻撃を披露。価値ある勝点3を手にした。
9,0,1,2-3,神戸はボールを保持し、パスを回して左サイドへ集める。橋本の果敢なオーバーラップから好機に結び付けるが、逆に空いた自陣のスペースを活用されてしまう。シンプルに速い攻撃を仕掛ける浦和に先制を許して折り返すと、後半はルーキーの佐々木が逆転ゴールを奪い、岩波の恩返し弾が飛び出すなど打ち合いの展開に。このまま引き分けかと思われたが、土壇場でマウリシオに決められて敗戦を喫した。対して連勝を挙げた浦和も、若い力が躍動。右サイドでスプリントを繰り返した橋岡のプレーは、生まれ変わったチームの象徴となった。


,results,home,score,match review
0,1,1,1-0,復活を目指す広島と、さらなるステップアップを見据える札幌。ともに新しい指揮官を迎えて臨んだ一戦は、充実の内容を見せた。広島は柏と川辺の両サイドを中心に主導権を握ると、前半28分にJリーグデビュー戦のティーラシンが先制点を奪う。その後は守勢に回る時間が続いたが、ボールホルダーへのプレスを徹底。90分間走り抜いて勝点3を手に入れた。一方、良い攻めを見せたものの最後の精度が足りず1点に泣いた札幌。だが、ピッチを広く使って攻め立てる「ミシャサッカー」の浸透ぶりを随所に見せ、今後への期待を持たせた。
1,1,0,1-2,浦和は立ち上がりからしっかりとパスを回してボールを握ると、右サイドからはマルティノスが、左サイドからは宇賀神が好機を演出する。前半終了間際には人数を掛けた攻撃を見せ、最後は青木がヘディングシュートを決めて先制。後半に入ってもアタッキングサードに持ち込む場面を多く作り、今季初勝利へサポーターの期待は高まった。しかし、追加点を奪えないことが明暗を分ける。少ない決定機を得点に結び付けた相手に逆転負けを喫した。対して耐える時間が長くなった広島は粘り強い守備を最後まで貫き、開幕2連勝を飾った。
2,1,0,0-4,Nah
3,1,0,0-1,互いにインテンシティーが高く、攻守が目まぐるしく入れ替わる展開が繰り広げられる。その中で、鹿島は安西、広島は川辺がとりわけ目立った働きを披露。両チームともにチャンスを作りつつも、スコアレスで前半を終えた。後半に入り、ミスを突かれて失点を喫したホームチームは、徐々に攻撃の圧力を強めていく。だが、ゴール前を固く閉ざす相手を崩せない。結局「常勝軍団」のお株を奪われるような戦い方を最後まで遂行され、近年得意としてきた広島戦で勝点を落としてしまった。
4,1,1,2-1,Nah
5,2,1,0-0,両チームともに守備の集中力が最後まで途切れず、スコアレスドローに終わった。広島は昨季まで磐田に在籍していた川辺が幅広く動いて攻撃に関与。また、リーグ戦初出場となった大南の隙を突いて何度かゴールに迫る。だが、GKを脅かすような決定機は作れず、ホームの大声援に応えられなかった。一方、大黒柱の中村を欠いて臨んだ磐田はアダイウトンの単独突破などで好機を創出する。しかし、そのアダイウトンは終盤に負傷交代。今季公式戦全勝の広島を止めたことは収穫だが、後味の悪い勝点1となった。
6,1,0,0-1,勝点で並ぶ上位チーム同士の対決は序盤から落ち着いた展開となる。ホームの川崎Fは今季初先発の大久保が随所に光るプレーを見せるも、大島の不在が影響してか、時折中盤でボールが停滞。昨季のような攻撃的サッカーは鳴りを潜め、スコアレスのまま、時間が経過していく。試合が動いたのは後半40分。登里の負傷などアクシデントによって不安定になっていた時間帯に、CKから先制点を奪われる。それでも反撃を試みると、終了間際にネットを揺らす。しかし、疑惑の判定によって幻となり、ホイッスル。後味の悪い敗戦となった。
7,2,1,0-0,Nah
8,1,0,0-1,ACL敗退のショックを払拭したい柏であったが、前半17分にサイド攻撃から失点を喫してしまう。その後は、統率されたブロックに加え最終局面で体を投げ出して守る相手を前に苦戦。伊東や初先発となった瀬川のサイド攻撃が不発に終わり、迎えた後半は大谷の投入後に得点の予感こそさせたが、集中力を保った鉄壁の守備陣を最後まで攻略できず。リーグ戦への切り替えのためにも欲しかった勝点3を取り逃し、開幕戦以来の黒星。一方の広島は唯一の無敗をキープ。首位を快走している。
9,1,1,3-1,ボール回しにたけ、強固な守備を誇る横浜FMに対し、広島は奪ってからの鋭いカウンターや、意表を突くロングシュートで相手ゴールを脅かす。押し込まれる場面があっても、粘り強く耐える時は耐え、伝家の宝刀を抜くタイミングを計る。すると、ビハインドを背負って迎えた後半に、一挙に3得点を挙げて逆転に成功。昨季は苦しいシーズンを過ごしたが、開幕からの無敗を継続して首位を快走しており、この一戦でも強い広島が帰ってきたことを印象付けた。


,results,home,score,match review
0,2,1,1-1,鳥栖は開始早々にPKを獲得すると、田川が冷静に沈めてさい先よいスタートを切る。リードを得て戦いやすくなったホームチームは堅いブロックを自陣に敷き、ボールを奪えば素早くカウンターを仕掛ける盤石のゲーム展開を披露する。だが、決定機を逃し続けていると、終盤は攻撃的な選手を次々と送り込んできたアウェイチームに対してディフェンスラインをずるずると下げてしまい、後半42分には痛恨の同点弾を献上。90分を通して優位に立つ時間が長かっただけに、ベアスタで勝点2を落とすもったいない開幕戦となってしまった。
1,2,0,2-2,J1での記念すべき初のホーム開幕戦を迎えた長崎は、序盤から積極的な姿勢を見せる。前半2分に澤田が豪快にネットを揺らし、さい先よく先制に成功。同35分には加入後初先発を果たした鈴木にもゴールが生まれ、最高の形で試合を折り返す。しかし、後半15分に失点を喫してしまい、流れは完全に鳥栖へ傾く。投入したチェキュベックを最終ラインに配置し、逃げ切りを図るものの、同39分にセットプレーから同点に追い付かれてしまう。雨が降る中でもトラスタに駆け付けた大勢のサポーターに、勝利をプレゼントすることはできなかった。
2,0,0,1-0,Nah
3,1,0,1-2,今季いまだリーグ戦での勝利がない両チームの一戦は、ヒートアップする場面が見られたように互いに攻守両面において球際で激しくボールを奪い合う展開となった。今節がホーム開幕戦となった横浜FMであったが、鳥栖の前線からの鋭いプレスに苦戦。パスサッカーも影を潜め、決定的な場面を作れずにいると、自陣でのパスミスから得点を許してしまう。後半はビクトルイバルボの力強い突破を中心とした相手攻撃陣への対応に手を焼き、後半12分にPKで追加点を奪われる。終盤に1点を返すも、今季初勝利を手にすることはできなかった。
4,0,1,0-2,Nah
5,0,1,0-1,互角の戦いを見せた鳥栖だったが、鹿島のエース金崎のゴールに沈んだ。前半から両チームともに球際が激しく、チャンスの応酬となる。鳥栖はショートカウンターの形から失点を喫したが、後半はビクトルイバルボにロングボールを集めることで多くの決定機を創出。何度もゴールに迫るが、相手守備陣の好守に阻まれ、あと一歩届かなかった。一方の鹿島は相手にペースを握られる時間帯が続くも、チーム全体で耐えて勝点3を獲得。特に、ビクトルイバルボを抑え込んだ昌子と植田のパフォーマンスは素晴らしく、代表戦に弾みをつけた。
6,1,1,3-2,公式戦2連敗中の鳥栖は、序盤から自陣で相手にボールを保持される苦しい展開に。前半14分にセットプレーの流れからガブリエルシャビエルの個人技に屈して先制を許すと、後半24分にはミスから追加点を奪われてしまう。しかし、ホームチームは諦めずに反撃を開始。高橋秀が1点を返し、その3分後にはビクトルイバルボのゴールで同点に追い付く。さらに攻勢を強めると、途中出場の田川が土壇場でネットを揺らし、逆転に成功。最後まで声援を送ったサポーターに勝点3をプレゼントした。
7,0,0,3-2,Nah
8,0,0,2-1,C大阪は序盤から相手のロングボールに対応し、ビクトルイバルボと田川の2トップを自由にプレーさせない。一方の攻撃面では、左サイドのスペースを狙ってチャンスメイク。高木や丸橋がサイドに大きく開いてボールを受け、クロスからゴールを脅かす。この攻撃が実を結んだのは前半27分。高木のクロスの流れから、最後は柿谷がゴールに流し込んで先制に成功した。後半10分にはセットプレーから追加点を奪取し、ゲームを優位に進める。終盤に1点を返されるも、大崩れすることはなく、終始安定した戦いを見せて勝点3を獲得した。
9,0,1,1-2,鳥栖は序盤から柏の鋭い出足に対応し切れず、サイドから崩されて先制点を奪われてしまう。それでも、以降はセカンドボールを拾えるようになり、ボールを落ち着かせていく。ところが、2トップへのパスの精度を欠き、決定機を創出できない。攻めあぐねる中で追加点を奪われ、苦しい展開で折り返す。後半に入り、安庸佑を投入すると、攻撃の選択肢が増え、立て続けにチャンスが生まれる。小野の豪快なシュートで1点を返し、その後も何度もゴールに迫り続ける。しかし同点ゴールは遠く、紙一重の差で逃げ切られ、連敗を喫してしまった。


,results,home,score,match review
0,0,0,2-1,昨季圧倒的なチーム力でJ2を制し、2年ぶりにJ1に帰還した湘南。開始早々に松田の力強い突破から新戦力のイジョンヒョプが先制点を奪い、そのままゲームを優位に進めるかと思われた。ところが、前半16分に追い付かれて以降、相手のコンパクトな守備を前に攻撃陣が沈黙。重苦しい空気が漂ったまま時間が経過してしまう。だが、そんな状況を打破したのは得意とするセットプレーだった。絶好の位置でFKを得ると、秋野のキックから最後は石川が押し込んで後半35分に勝ち越しに成功。少ないチャンスを確実に生かし、勝利を収めた。
1,2,1,2-2,J1での記念すべき初のホーム開幕戦を迎えた長崎は、序盤から積極的な姿勢を見せる。前半2分に澤田が豪快にネットを揺らし、さい先よく先制に成功。同35分には加入後初先発を果たした鈴木にもゴールが生まれ、最高の形で試合を折り返す。しかし、後半15分に失点を喫してしまい、流れは完全に鳥栖へ傾く。投入したチェキュベックを最終ラインに配置し、逃げ切りを図るものの、同39分にセットプレーから同点に追い付かれてしまう。雨が降る中でもトラスタに駆け付けた大勢のサポーターに、勝利をプレゼントすることはできなかった。
2,2,0,2-2,Nah
3,2,1,1-1,J1初勝利を挙げたい長崎。浦和にボールを支配される時間帯が続くが、ファンマのキープ力を起点にして攻撃を仕掛ける。前半44分には、翁長が遠藤に倒されて得たPKをこのゲームでJ1通算100試合出場を達成した鈴木が確実に決め、リードを奪って前半を終了。後半に入っても守ってカウンター攻撃を仕掛けるスタイルは変えずに粘り強く戦ったものの、後半31分に左CKからズラタンに同点弾を決められてしまう。その後も勝利を目指してチャンスは創出したが、最後まで勝ち越し点を挙げられず。前節に続いて勝点3を逃してしまった。
4,1,1,2-1,Nah
5,0,0,2-1,今季初勝利を狙う札幌。パスをつなぎながら両サイドからのクロスとジェイの高さを起点にして攻撃を仕掛けるが、得点を奪えぬまま試合を折り返す。後半に入ると、チャナティップが躍動。後半8分に体の強さを生かしてDFをかわし、ジェイに縦パスを送ると、ジェイがこれを決めて先制する。同点で迎えた同49分には、ジェイが頭で折り返したボールに反応して劇的なヘディングシュートを決め、今季初の勝点3に大きく貢献した。対する長崎は、一時は同点に追い付いたが、逆転ゴールを奪えず。次節でJ1初勝利を挙げたい。
6,0,0,1-0,今季いまだ無敗と好調を維持する仙台がまたしても「ウノゼロ」の勝利を挙げた。立ち上がりこそ相手の素早いプレスに加えて鈴木、ファンマの裏への抜け出しに手を焼き、押し込まれる時間帯が続いたが、それでも耐えられるのが今季のホームチーム。しっかり相手の攻撃をシャットアウトすると、今季初先発の西村が相手のクリアミスを見逃さず、ユアスタを沸かす。その後もバランスよくゲームをコントロールし、相手の攻撃を抑え込んだ。一方の長崎はチャンスをモノにできず。またしてもJ1初勝利はお預けとなった。
7,1,1,3-2,Nah
8,0,1,2-5,長崎は開始直後の失点で出はなをくじかれると、その後も自陣の左サイドのスペースを活用されてピンチを招くなど、ペースをつかめない。ボールを保持した状態であっても、味方のパスを引き出す動きが乏しく、ロングボールに頼ることが多くなり、個々の奮闘に頼るシーンが頻出。セットプレーとショートカウンターで2点を奪うものの、リーグ戦初勝利には届かなかった。対するFC東京は、個人技や連係で一枚上手であることを見せつける。ディエゴオリヴェイラがハットトリックを達成し、3連勝を飾った。
9,1,0,0-1,長崎が歴史的勝利を手にした。序盤からお互いに攻めあぐねる状況だったが、鈴木が均衡を破る先制弾を奪取。1点のリードを保って前半を折り返すと、後半はホームチームが前線の選手を3枚投入して攻勢を強める。クリスラン、ミッチェルデューク、北川に度々攻め込まれるが、運動量で引けを取らないアウェイチームは何度もその攻撃をはね返した。終盤のパワープレーを全員守備でしのぐと、ここで試合終了のホイッスル。アイスタでJ1史上初の勝点3を手にし、選手、スタッフ、サポーターが一体となって歓喜に沸いた。
